In [ ]:
# 查看数据文件目录  list datalab files
!ls datalab/

In [ ]:
# 查看个人永久空间文件  list files in your permanent storage
!ls /home/tianchi/myspace/


In [ ]:
# 查看当前kernel下已安装的包  list packages
!pip list --format=columns

In [1]:
from collections import defaultdict
import random
import numpy as np
import torch
from torch import nn

In [2]:
class WordEmbCbow(nn.Module):
    def __init__(self,nwords,emb_size):
        super(WordEmbCbow,self).__init__()
        self.emb=nn.Embedding(nwords,emb_size)
        nn.init.uniform_(self.emb.weight,-0.25,0.25)
        self.out=nn.Linear(emb_size,nwords)
        nn.init.uniform_(self.out.weight,-0.25,0.25)
    def forward(self,words):
        emb=self.emb(words)
        emb_sum=torch.sum(emb,dim=0)
        emb_new=emb_sum.view(1,-1)
        out=self.out(emb_new)
        return out
        
        

In [ ]:
N=2
EMB_SIZE=128
w2i=defaultdict(lambda:len(w2i))
UNK=w2i["<unk>"]
S = w2i["<s>"]#开始和结束的标志
label_location="out/label.txt"
embeddings_location="out/embedding.txt"

def read_dataset(filename):
    with open(filename,"r") as f:
        for line in f:
            yield[w2i[i] for i in line.strip().split(" ")]

            
train=list(read_dataset("Demo/DataSets/train.txt"))
w2i=defaultdict(lambda :UNK,w2i)
dev=list(read_dataset("Demo/DataSets/test.txt"))
nwords=len(w2i)
i2w={v:k for k,v in w2i.items()}

with open(label_location,"w") as f:#将字典存储起来
    for i in range(nwords):
      f.write(i2w[i]+"\n")#字典中的一个词占一行

model=WordEmbCbow(nwords,EMB_SIZE)
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
def sent_loss(sent):
    losses=[]
    new_sent=[S]*N+sent+[S]*N
    for i in range(N,len(sent)+N):
            c=torch.tensor(new_sent[i-N:i]+new_sent[i+1:i+N+1])
            t=torch.tensor([new_sent[i]])
            logits=model(c)
            loss=criterion(logits,t)
            losses.append(loss)
    return torch.stack(losses).sum()
for epoch in range(100):
    random.shuffle(train)
    train_words,train_loss=0,0.0
    for sent_id,sent in enumerate(train):
        my_loss=sent_loss(sent)#总损失
        train_loss+=my_loss.item()
        train_words+=len(sent)
        optimizer.zero_grad()
        my_loss.backward()
        optimizer.step()
        if(sent_id%5==0):
            print("epoch=%r,loss=%.4f"%(epoch,train_loss/train_words))
    print("epoch=%r,loss=%.4f"%(epoch,train_loss/train_loss))
    model.eval()
    dev_words,dev_loss=0,0.0
    for sent_id,sent in enumerate(dev):
        my_loss=sent_loss(sent)#总损失
        dev_loss+=my_loss.item()
        dev_words+=len(sent)
        optimizer.zero_grad()
        my_loss.backward()
        optimizer.step()
        if(sent_id%500==0):
            print("epoch=%r,loss=%.4f"%(epoch,dev_loss/dev_words))
    print("epoch=%r,loss=%.4f"%(epoch,dev_loss/dev_loss))

with open(embeddings_location, 'w') as embeddings_file:
        W_w_np = model.emb.weight.data.numpy()
        for i in range(nwords):#第几个单词
            ith_embedding = '\t'.join(map(str, W_w_np[i]))
            embeddings_file.write(ith_embedding + '\n')




epoch=0,loss=9.4405
epoch=0,loss=9.3011
epoch=0,loss=9.1932
epoch=0,loss=9.1243
epoch=0,loss=8.9659
epoch=0,loss=8.9011
epoch=0,loss=8.7007
epoch=0,loss=8.6057
epoch=0,loss=8.5507
epoch=0,loss=8.4961
epoch=0,loss=8.4243
epoch=0,loss=8.3348
epoch=0,loss=8.1396
epoch=0,loss=8.1230
epoch=0,loss=8.0659
epoch=0,loss=8.0511
epoch=0,loss=8.0369
epoch=0,loss=8.0303
epoch=0,loss=7.9857
epoch=0,loss=7.9765
epoch=0,loss=7.9605
epoch=0,loss=7.9326
epoch=0,loss=7.9040
epoch=0,loss=7.8868
epoch=0,loss=7.8792
epoch=0,loss=7.8311
epoch=0,loss=7.8159
epoch=0,loss=7.7865
epoch=0,loss=7.7603
epoch=0,loss=7.7496
epoch=0,loss=7.7542
epoch=0,loss=7.7554
epoch=0,loss=7.7497
epoch=0,loss=7.7212
epoch=0,loss=7.7064
epoch=0,loss=7.7138
epoch=0,loss=7.7287
epoch=0,loss=7.7260
epoch=0,loss=7.7087
epoch=0,loss=7.7033
epoch=0,loss=7.6905
